# Credit Ratings on Earnings Call Dates

Use earnings call data (symbol and date for companies) and credit ratings data at the level of issuances to create a dataset of credit ratings on earnings call dates.

In [10]:
# packages
import pandas as pd
import pandasql as ps

In [11]:
# Load credit ratings data
# C:\Users\ijyli\Box\STAT 222 Capstone\Intermediate Data\combined_credit_rating_data.csv
credit_ratings = pd.read_csv('~/Box/STAT 222 Capstone/Intermediate Data/combined_credit_rating_data.csv')
# Rename Date to rating_date, Next Rating Date or End of Data to next_rating_date_or_end_of_data, Symbol to ticker
credit_ratings = credit_ratings.rename(columns={'Date': 'rating_date', 'Next Rating Date or End of Data': 'next_rating_date_or_end_of_data', 'Symbol': 'ticker'})
# Create year variable from rating_date
credit_ratings['Year'] = pd.to_datetime(credit_ratings['rating_date']).dt.year
# Limit rating agency and scope
credit_ratings = credit_ratings[credit_ratings['Rating Agency Name'].str.contains('Standard & Poor\'s') & (credit_ratings['Year'] >= 2010) & (credit_ratings['Year'] <= 2016)]
credit_ratings

,Rating,ticker,Rating Agency Name,rating_date,Source,Rating Rank AAA is 10,Next Rating,Next Rating Date,Previous Rating,Previous Rating Date,next_rating_date_or_end_of_data,Type,Change in Rating,Year
5560,AAA,AAPL,Standard & Poor's Ratings Services,2014-04-24,Supplementary,10,AA,2014-05-27,NaN,NaN,2014-05-27,Ambiguous,NaN,2014
5561,AA,AAPL,Standard & Poor's Ratings Services,2014-05-27,Supplementary,9,AA,2015-02-18,AAA,2014-04-24,2015-02-18,Downgrade,-1.0,2014
5562,AA,AAPL,Standard & Poor's Ratings Services,2015-02-18,Supplementary,9,AA,2015-05-28,AA,2014-05-27,2015-05-28,Affirmation,0.0,2015
5563,AA,AAPL,Standard & Poor's Ratings Services,2015-05-28,Both,9,AA,2015-06-02,AA,2015-02-18,2015-06-02,Affirmation,0.0,2015
5564,AA,AAPL,Standard & Poor's Ratings Services,2015-06-02,Supplementary,9,AA,2015-08-25,AA,2015-05-28,2015-08-25,Affirmation,0.0,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8727,BBB,ZTS,Standard & Poor's Ratings Services,2014-01-31,Both,7,BBB,2015-01-30,BBB,2013-10-11,2015-01-30,Affirmation,0.0,2014
8728,BBB,ZTS,Standard & Poor's Ratings Services,2015-01-30,Both,7,BBB,2015-11-03,BBB,2014-01-31,2015-11-03,Affirmation,0.0,2015
8729,BBB,ZTS,Standard & Poor's Ratings Services,2015-11-03,Supplementary,7,BBB,2016-01-22,BBB,2015-01-30,2016-01-22,Affirmation,0.0,2015
8730,BBB,ZTS,Standard & Poor's Ratings Services,2016-01-22,Both,7,BBB,2016-12-23,BBB,2015-11-03,2016-12-23,Affirmation,0.0,2016


In [12]:
# Load selected variables from earnings call data
earnings_call_dates = pd.read_csv('~/Box/STAT 222 Capstone/Intermediate Data/call_for_merge.csv')
# Rename company to ticker
# Rename date to earnings_call_date
# Keep only these columns
earnings_call_dates = earnings_call_dates.rename(columns={'company': 'ticker', 'date': 'earnings_call_date'})
earnings_call_dates = earnings_call_dates[['ticker', 'earnings_call_date']]
earnings_call_dates

,ticker,earnings_call_date
0,ADNT,2017-04-28
1,ADNT,2017-07-27
2,ADNT,2017-11-05
3,ADNT,2018-01-29
4,ADNT,2018-05-03
...,...,...
62069,KMX,2021-12-22
62070,KMX,2022-04-12
62071,KMX,2022-06-24
62072,KMX,2022-09-29


## Join together

In [13]:
query = '''
SELECT a.*, b.earnings_call_date
FROM credit_ratings a
INNER JOIN earnings_call_dates b
ON a.ticker = b.ticker
    AND a.rating_date <= b.earnings_call_date
        AND b.earnings_call_date <= a.next_rating_date_or_end_of_data
'''

# Run query
credit_ratings_on_earnings_call_dates = ps.sqldf(query, locals())

In [14]:
# Check for uniqueness by ticker and earnings_call_date
ec_rating_cnt = credit_ratings_on_earnings_call_dates.groupby(['ticker', 'earnings_call_date']).size().reset_index(name='count').sort_values(by='count', ascending=False)
ec_rating_cnt[ec_rating_cnt['count'] > 1].sort_values(by=['earnings_call_date'])

,ticker,earnings_call_date,count
1259,EMN,2012-01-27,2
895,CRK,2012-02-07,2
3911,STZ,2012-06-29,2
1062,DHI,2013-01-29,2
2017,HST,2013-02-21,2
3632,SBGI,2013-04-29,2
696,CENX,2013-07-31,2
3343,PKG,2013-10-15,2
2434,LEA,2014-01-31,2
4184,TRTN,2014-07-24,2


In [15]:
# We have cases with rating and earnings call on the same day
# In these cases, let's assume the rating occurred before the earnings call
# Therefore, that call will inform the rating on the next earnings call date, not the one on the same day
# Require earnings_call_date not equal to next_earnings_call_date
credit_ratings_on_earnings_call_dates = credit_ratings_on_earnings_call_dates[credit_ratings_on_earnings_call_dates['earnings_call_date'] != credit_ratings_on_earnings_call_dates['next_rating_date_or_end_of_data']]

In [16]:
# Check for uniqueness by ticker and earnings_call_date
ec_rating_cnt = credit_ratings_on_earnings_call_dates.groupby(['ticker', 'earnings_call_date']).size().reset_index(name='count').sort_values(by='count', ascending=False)
ec_rating_cnt[ec_rating_cnt['count'] > 1].sort_values(by=['earnings_call_date'])

,ticker,earnings_call_date,count


## Credit Rating One Call Ahead, Two Calls Ahead, etc.

In [17]:
# Sort by ticker and earnings_call_date
credit_ratings_on_earnings_call_dates = credit_ratings_on_earnings_call_dates.sort_values(by=['ticker', 'earnings_call_date'])

# Get next earnings call date and next value of rating
credit_ratings_on_earnings_call_dates['next_earnings_call_date'] = credit_ratings_on_earnings_call_dates.groupby('ticker')['earnings_call_date'].shift(-1)
credit_ratings_on_earnings_call_dates['rating_on_next_earnings_call_date'] = credit_ratings_on_earnings_call_dates.groupby('ticker')['Rating'].shift(-1)

# Helpful difference variable of days until next earnings call
credit_ratings_on_earnings_call_dates['days_until_next_earnings_call'] = (pd.to_datetime(credit_ratings_on_earnings_call_dates['next_earnings_call_date']) - pd.to_datetime(credit_ratings_on_earnings_call_dates['earnings_call_date'])).dt.days

# Two earnings call dates later
# lead_2_earnings_call_date
credit_ratings_on_earnings_call_dates['lead_2_earnings_call_date'] = credit_ratings_on_earnings_call_dates.groupby('ticker')['earnings_call_date'].shift(-2)
# lead_2_rating
credit_ratings_on_earnings_call_dates['lead_2_rating'] = credit_ratings_on_earnings_call_dates.groupby('ticker')['Rating'].shift(-2)

## Output CSV

In [18]:
# Reorder columns - ticker, earnings_call_date, next_earnings_call_date, rating_on_next_earnings_call_date, days_until_next_earnings_call, then all other columns
# Columns to move to the front
cols_to_front = ['ticker', 'earnings_call_date', 'next_earnings_call_date', 'rating_on_next_earnings_call_date', 'days_until_next_earnings_call']
# Reorder columns
new_columns_order = cols_to_front + [col for col in credit_ratings_on_earnings_call_dates.columns if col not in cols_to_front]
credit_ratings_on_earnings_call_dates = credit_ratings_on_earnings_call_dates[new_columns_order]

# Save to csv
credit_ratings_on_earnings_call_dates.to_csv('~/Box/STAT 222 Capstone/Intermediate Data/credit_ratings_on_earnings_call_dates.csv', index=False)